# Imports and environment

In [42]:
import os
import sys
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI
from markdown_pdf import MarkdownPdf, Section

# Load environment variables (e.g. OPENAI_API_KEY)
load_dotenv()

True

# User‐configurable variables (formerly CLI args)

In [51]:
output_file   = "../results/SnowPro Advanced Data Engineer Prep.pdf"           # --output
model         = "gpt-4.1-mini"         # --model
toc_level     = 3                      # --toc-level
optimize      = False                  # --optimize
use_web_search= True                   # --no-web-search → False
verbose       = False                  # --verbose

# Helper function definitions

In [ ]:
def generate_markdown_from_prompt(user_prompt, client):
    try:
        prompt = [
            {"role": "system", "content": (
                "You are a tutor preparing me to the SnowPro® Advanced: Data Engineer (DEA-C02) certification. "
                "Provide output in Markdown. "
                "Use **bold** text for headers and not don't use '#' or '#'s headers. "
                "Don't add line separators in response. "
                "Add code snippets where needed."
                ""
            )},
            {"role": "user", "content": user_prompt},
        ]
        tools = [{"type": "web_search_preview"}] if use_web_search else []
        response = client.responses.create(
            model=model,
            tools=tools,
            input=prompt,
        )
        text = f"Prompt: {user_prompt}\n\n" + response.output_text.strip()
        return text
    except Exception as e:
        print("AI request failed for node '%s': %s" + user_prompt + e)
        text(user_prompt + "\n\n---\n\n")
        return text

def process_prompts(prompts_text, client, pdf, css):
    lines = prompts_text.strip().splitlines()
    
    len_lines = len(lines)
    
    section_md = ""
    for idx, line in enumerate(lines, start=1):
        stripped = line.lstrip()
        # skip empty lines
        if not stripped:
            continue
        # skip lines starting with '#'
        elif stripped.startswith("#"):
            section_md = f"{section_md}\n\n{stripped}"
        else:
            print(f"Processing Line {idx}/{len_lines} ({round(idx*100/len_lines)}%)...")
            md = generate_markdown_from_prompt(stripped, client)
            section_md = f"{section_md}\n\n{md}\n\n"
            pdf.add_section(Section(section_md), user_css=css)
            section_md = ""

# Prompts

In [ ]:
chatgpt_prompts = """
# SNOWPROⓇ ADVANCED: DATA ENGINEER (DEA-C02)
## Domain 1.0: Data Movement (20 Questions)

1. What Snowflake stages can you load data from, and when would you choose each?
2. How does Snowflake’s micro-partitioning affect bulk data loading performance?
3. Describe the impact of file format options (e.g., CSV vs. JSON) on COPY INTO behavior.
4. What are best practices for designing a Snowflake FILE FORMAT for semi-structured data?
5. How do you configure automatic schema inference when loading JSON files?
6. Explain how to validate a data load and identify failed rows using VALIDATE and COPY_HISTORY.
7. What causes a “data conversion” error during ingestion, and how would you resolve it?
8. How do internal, external, and named stages differ in Snowflake, and what are their use cases?
9. Compare Auto-ingest Snowpipe versus REST API Snowpipe: pros, cons, and triggers.
10. Describe the role of Snowpipe Streaming and how it differs from standard Snowpipe.
11. How would you design a streaming pipeline using Streams and Tasks to capture incremental changes?
12. What are common errors when using a CREATE STREAM and how can you troubleshoot them?
13. When ingesting XML data, what file format and parsing options are essential?
14. How do you implement a continuous data pipeline with Snowpark for Python?
15. Outline the steps to build a Kafka connector to push data into Snowflake.
16. What configuration options exist for the Spark Connector when unloading data back to Parquet?
17. How does Snowflake’s Python Connector handle large-scale data loads differently from COPY INTO?
18. Explain how to set up secure data sharing with row-level security using row access policies.
19. When should you use an external table versus loading data into a managed table?
20. How do you unload partitioned Parquet files to S3 using COPY INTO with PATTERN matching?

## Domain 2.0: Performance Optimization (20 Questions)

21. How can QUERY_HISTORY and SYSTEM$CLUSTERING_INFORMATION be used to diagnose slow queries?
22. What factors determine whether to scale out (multi-cluster) vs. scale up (warehouse size)?
23. Explain how automatic result caching works and how to monitor cache hit rates.
24. When should you use a Materialized View, and what maintenance overhead does it introduce?
25. How does Search Optimization Service improve point queries, and what are its cost implications?
26. Describe how to interpret the EXPLAIN plan JSON to identify bottlenecks.
27. What query patterns benefit most from Query Acceleration Service?
28. How do clustering keys impact micro-partition pruning, and how do you measure clustering depth?
29. What best practices exist for warehouse auto-suspend and auto-resume to balance cost and performance?
30. How can you use ACCOUNT_USAGE views to track warehouse credit consumption trends?
31. Describe how to set up alerting on long-running tasks using CREATE ALERT and TASK_HISTORY.
32. What are common causes of disk spilling, and how can you prevent it?
33. How does a Snowpark-optimized warehouse differ in performance characteristics?
34. Explain how to monitor Snowpipe Streaming throughput and troubleshoot backlogs.
35. When is it appropriate to use persistent result caching over temporary table storage?
36. What metrics in the LOAD_HISTORY view help you identify performance issues in data loads?
37. How can you use system functions like SYSTEM$PIPE_STATUS to track pipeline health?
38. Describe a troubleshooting workflow for a multi-cluster warehouse that refuses to scale out.
39. How do you use QUERY_ACCELERATION_ON for a specific warehouse via ALTER WAREHOUSE?
40. Explain the impact of high concurrency on virtual warehouse queuing and credit usage.

## Domain 3.0: Storage & Data Protection (20 Questions)

41. How does Snowflake’s Time Travel retention period interact with Fail-Safe?
42. Describe the steps to perform an object clone and test changes before promotion.
43. How would you use Database Replication to implement cross-region disaster recovery?
44. What system views provide insight into micro-partition storage metrics?
45. How do you analyze clustering depth with SYSTEM$CLUSTERING_DEPTH?
46. Explain how to roll back a dropped table using Time Travel.
47. What are the considerations when adjusting TIME_TRAVEL_RETENTION_TIME_IN_DAYS at the account level?
48. How does Snowflake synchronize failover and failback for accounts?
49. When unloading data with COPY INTO, how can you ensure schema evolution is respected?
50. Describe the differences between object cloning and database replication in a CI/CD pipeline.
51. How do you monitor TABLE_STORAGE_METRICS to identify hot partitions?
52. Explain how to combine Streams with Time Travel to create auditing pipelines.
53. What are the best practices for setting micro-partition burst clustering?
54. How does cross-cloud replication differ from cross-region replication?
55. What happens to Streams on a cloned table, and how do you manage them post-clone?
56. How can you use system functions to estimate storage costs per table?
57. Describe the effect of Fail-Safe on long-term storage cost and data retention SLAs.
58. When is it appropriate to use continuous data protection features over standard Time Travel?
59. How do you set up and monitor account-level replication schedules using SHOW REPLICATION ACCOUNTS?
60. Explain the process of restoring an individual schema from a past point-in-time.

## Domain 4.0: Data Governance (20 Questions)

61. How do you apply and enforce object tagging for data classification at scale?
62. Describe how to implement column-level security using dynamic data masking.
63. What is the difference between a masking policy and a row access policy?
64. How do you audit role inheritance vs. role composition in RBAC?
65. Explain how to use SHOW OBJECT DEPENDENCIES to understand lineage.
66. When would you choose external tokenization over dynamic data masking?
67. How do you monitor data quality using DATA_QUALITY and METRIC functions?
68. Describe how to implement aggregation policies for PII-sensitive columns.
69. What API calls are required to automate Snowflake Data Clean Rooms setup?
70. How do you configure a projection policy for a high-sensitivity dataset?
71. Explain the best practices for creating and managing row access policies.
72. How can you use the ACCOUNT_USAGE.TAG_REFERENCES view to track tagging consistency?
73. Describe how to integrate Snowflake governance with external data lineage tools.
74. What is the role of ALERTS in governance, and how do you configure them for policy violations?
75. How do you manage dynamic data masking on VARIANT columns containing nested PII?
76. Explain how Snowflake Marketplace listings enforce sharing and revocation of data.
77. What governance considerations apply when using external stages to load sensitive data?
78. How do you leverage the SHOW MASKING POLICIES command for auditing?
79. When would you use the Snowflake developer API versus SnowCLI for governance tasks?
80. Describe how to encrypt data in transit and at rest in Snowflake to comply with GDPR.

## Domain 5.0: Data Transformation (20 Questions)

81. Compare SQL UDFs versus Snowpark UDFs in terms of performance and security.
82. How do you register and invoke a JavaScript UDF that processes JSON objects?
83. Describe the steps to create and use a Secure UDF for proprietary algorithms.
84. What are User-Defined Table Functions (UDTFs), and when would you use them?
85. Explain how to design a UDAF to compute session-based aggregations.
86. How do you set up an external function that calls an AWS Lambda endpoint?
87. What security configurations are required for secure external functions?
88. Describe how to write a Snowpark stored procedure in Scala for complex ETL logic.
89. How can you traverse deeply nested VARIANT data using FLATTEN and lateral joins?
90. Explain how to convert structured table data into JSON documents for NoSQL ingestion.
91. What is a directory table, and how do you query S3 bucket listings via Snowflake?
92. How do you use the Rest API integration to load unstructured files into Snowflake stages?
93. Compare SQL Scripting stored procedures with JavaScript stored procedures for orchestration.
94. Describe transaction control (BEGIN, COMMIT, ROLLBACK) in Snowflake Scripting.
95. How do you use Snowpark DataFrames to perform an incremental upsert into a target table?
96. What best practices exist for managing large-scale UDF deployments across multiple schemas?
97. Explain how TRY_PARSE_JSON helps with semi-structured data ingestion.
98. How can you profile and optimize Snowpark DataFrame transformations for large datasets?
99. Describe how to implement slow-changing dimensions using Streams and Tasks in Snowpark.
100. How do you orchestrate a multi-step data transformation pipeline entirely within Snowflake?
"""

<>:1: SyntaxWarning: invalid escape sequence '\_'
<>:1: SyntaxWarning: invalid escape sequence '\_'
C:\Users\Kallibek\AppData\Local\Temp\ipykernel_24972\4231384259.py:1: SyntaxWarning: invalid escape sequence '\_'
  chatgpt_prompts = """


# Instantiate client, load YAML, and generate Markdown sections

In [47]:
from pathlib import Path

api_key = os.getenv("OPENAI_API_KEY")
css = Path("custom.css").read_text(encoding="utf-8")
pdf = MarkdownPdf(toc_level=toc_level, optimize=optimize)

if not api_key:
    print("Environment variable OPENAI_API_KEY is not set.")
    sys.exit(1)

client = OpenAI(api_key=api_key)
            
process_prompts(chatgpt_prompts, client, pdf, css)


Processing Line 4/115 (3%)...
Processing Line 5/115 (4%)...
Processing Line 6/115 (5%)...
Processing Line 7/115 (6%)...
Processing Line 8/115 (7%)...
Processing Line 9/115 (8%)...
Processing Line 10/115 (9%)...
Processing Line 11/115 (10%)...
Processing Line 12/115 (10%)...
Processing Line 13/115 (11%)...
Processing Line 14/115 (12%)...
Processing Line 15/115 (13%)...
Processing Line 16/115 (14%)...
Processing Line 17/115 (15%)...
Processing Line 18/115 (16%)...
Processing Line 19/115 (17%)...
Processing Line 20/115 (17%)...
Processing Line 21/115 (18%)...
Processing Line 22/115 (19%)...
Processing Line 23/115 (20%)...
Processing Line 27/115 (23%)...
Processing Line 28/115 (24%)...
Processing Line 29/115 (25%)...
Processing Line 30/115 (26%)...
Processing Line 31/115 (27%)...
Processing Line 32/115 (28%)...
Processing Line 33/115 (29%)...
Processing Line 34/115 (30%)...
Processing Line 35/115 (30%)...
Processing Line 36/115 (31%)...
Processing Line 37/115 (32%)...
Processing Line 38/11

KeyboardInterrupt: 

# Save as PDF

In [52]:
try:
    pdf.save(output_file)
    print(f"PDF successfully saved to {output_file}")
except Exception as e:
    print(f"Failed to save PDF: {e}")
    raise

PDF successfully saved to ../results/SnowPro Advanced Data Engineer Prep.pdf
